In [68]:
import datetime as dt
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import requests
from bs4 import BeautifulSoup
import re
import datetime as dt

import os
import json

from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [26]:
INSTIZ_URL = 'https://www.instiz.net/name_enter'
INSTIZ_HEADERS = { 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'}

In [93]:
def fetch_posts_by_date_range(start_date, end_date):
    base_url = 'https://www.instiz.net/name_enter/'
    links = []
    current_page = 959
    continue_search = True
    driver = webdriver.Chrome()

    # 날짜 형식을 datetime 객체로 변환
    start_date = dt.datetime.strptime(start_date, "%Y-%m-%d").date()
    end_date = dt.datetime.strptime(end_date, "%Y-%m-%d").date()
    today = dt.datetime.now().date()

    try:
        while continue_search:
            url = f"{base_url}?page={current_page}"
            driver.get(url)
            
            # 페이지가 완전히 로드될 때까지 대기
            WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'tr[id="detour"]')))
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            rows = soup.find_all('tr', id="detour")

            for row in rows:
                date_td = row.find('td', class_='listno regdate')
                date_str = date_td.text.strip() if date_td else None
                no_value = date_td.get('no') if date_td else None

                if date_str and no_value:
                    try:
                        if ':' in date_str and len(date_str) <= 5:  # 시간:분 형태로 표시된 오늘 게시글
                            post_date = today
                        else:  # yyyy.mm.dd hh:mm 형식의 날짜
                            date_str = f"{today.year}.{date_str}"
                            post_date = dt.datetime.strptime(date_str, "%Y.%m.%d %H:%M").date()

                        if start_date <= post_date <= end_date:
                            link = base_url + no_value
                            links.append(link)
                            print(f"Added link: {link}")
                        elif post_date < start_date:
                            continue_search = False
                            break
                    except ValueError:
                        continue  # 날짜 형식이 맞지 않으면 다음 row로 넘어감

            print(f"Processed page {current_page}, total links collected: {len(links)}")
            
            if continue_search:
                current_page += 1  # 다음 페이지로 이동
            else:
                print("Search has exited: Reached earlier than start date.")
                break

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        driver.quit()  # Ensure the driver is quit properly

    return links

In [82]:
def get_instiz_content(link):
    resp = requests.get(link, headers = INSTIZ_HEADERS)
    resp.raise_for_status()
    resp.encoding = 'utf-8'

#     resp.encoding = None
    html = resp.text
    soup = BeautifulSoup(html, 'html.parser')

    post_id = link.split('/')[-1]
#     post_id = re.search(r'/(\d+)\?', link).group(1)
    community = 'INSTIZ'
    category = '연예'
    title_element = soup.find('span', id='nowsubject')
    title = ''.join([str(content) for content in title_element.contents if content.name is None]).strip()
    vote_up_element = soup.find('span', class_=re.compile(r'^votenow\d+'))
    vote_up = int(vote_up_element.text.strip()) if vote_up_element and vote_up_element.text.strip().isdigit() else None
    vote_down = None
    view = int(soup.find('span', id='hit').text.strip())
    writer = None
    date_str = soup.find('span', {'itemprop': 'datePublished'})['title'] if soup.find('span', {'itemprop': 'datePublished'}) else None
    date = dt.datetime.strptime(date_str, '%Y/%m/%d %H:%M:%S').strftime('%Y-%m-%d %H:%M:%S')
    content = soup.find('div', {'id': 'memo_content_1'}).text.strip()
    comment_elements = soup.find_all('td', class_='comment_memo')
    comments = []

    for element in comment_elements:
        # 원 댓글 찾기
        main_comment = element.find('span', {'id': re.compile(r'^n\d+')})
        if main_comment:
            comments.append(main_comment.text.strip())

    # 결과 출력
    data = {
        'id': 'INTZ'+post_id,
        'community': community,
        'link': link,
        'category': category,
        'title': title,
        'vote_up': vote_up,
        'vote_down': vote_down,
        'view': view,
        'writer': writer,
        'date': date,
        'content': content,
        'comments': comments,
        'ncomment' : len(comments),
        'timestamp': dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    return data 

In [102]:
def retry_failed_links(failed_file, output_file, existing_ids, all_content):
    try:
        # 실패한 링크 파일 열기
        with open(failed_file, "r", encoding="utf-8") as f:
            failed_links = json.load(f)
        
        # 재시도 실패한 링크 저장용
        failed_links_retry = []
        
        for idx, link in enumerate(failed_links, start=1):
            try:
                content = get_instiz_content(link)
                if content['id'] not in existing_ids:  # 중복 확인
                    all_content.append(content)
                    existing_ids.add(content['id'])  # 중복 방지를 위해 추가된 ID를 기록
                    print(f"[Retry {idx}/{len(failed_links)}] Successfully processed: {link}")
                else:
                    print(f"[Retry {idx}/{len(failed_links)}] Duplicate ID, skipped: {content['id']}")
            except Exception as e:
                print(f"[Retry {idx}/{len(failed_links)}] Error occurred while processing {link}: {e}")
                failed_links_retry.append(link)  # 재시도 실패한 링크 저장

        # 모든 데이터 처리 후 저장
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(all_content, f, indent=4, ensure_ascii=False)
        print(f"\nAll records saved to {output_file} (Total records: {len(all_content)})")

        # 재시도 실패한 링크 저장
        if failed_links_retry:
            with open("failed_links_retry.json", "w", encoding="utf-8") as f:
                json.dump(failed_links_retry, f, indent=4, ensure_ascii=False)
            print(f"Retry failed links saved to failed_links_retry.json (Total failed: {len(failed_links_retry)})")
    except FileNotFoundError:
        print(f"No failed links file found: {failed_file}")

In [95]:
start_date = "2024-11-02"
end_date = "2024-11-05" #5일부터해야됨

info = fetch_posts_by_date_range(start_date, end_date)
failed_links = []  # 에러가 발생한 링크를 저장할 리스트
total_links = len(info)

Processed page 959, total links collected: 0
Processed page 960, total links collected: 0
Processed page 961, total links collected: 0
Processed page 962, total links collected: 0
Processed page 963, total links collected: 0
Processed page 964, total links collected: 0
Processed page 965, total links collected: 0
Processed page 966, total links collected: 0
Processed page 967, total links collected: 0
Processed page 968, total links collected: 0
Processed page 969, total links collected: 0
Processed page 970, total links collected: 0
Processed page 971, total links collected: 0
Processed page 972, total links collected: 0
Processed page 973, total links collected: 0
Processed page 974, total links collected: 0
Processed page 975, total links collected: 0
Processed page 976, total links collected: 0
Processed page 977, total links collected: 0
Added link: https://www.instiz.net/name_enter/94064362
Added link: https://www.instiz.net/name_enter/94064358
Added link: https://www.instiz.net/

Added link: https://www.instiz.net/name_enter/94064019
Added link: https://www.instiz.net/name_enter/94064018
Added link: https://www.instiz.net/name_enter/94064017
Added link: https://www.instiz.net/name_enter/94064016
Added link: https://www.instiz.net/name_enter/94064014
Added link: https://www.instiz.net/name_enter/94064013
Added link: https://www.instiz.net/name_enter/94064009
Added link: https://www.instiz.net/name_enter/94064007
Added link: https://www.instiz.net/name_enter/94064005
Added link: https://www.instiz.net/name_enter/94064004
Added link: https://www.instiz.net/name_enter/94064001
Added link: https://www.instiz.net/name_enter/94063999
Added link: https://www.instiz.net/name_enter/94063993
Added link: https://www.instiz.net/name_enter/94063990
Added link: https://www.instiz.net/name_enter/94063989
Added link: https://www.instiz.net/name_enter/94063986
Added link: https://www.instiz.net/name_enter/94063983
Added link: https://www.instiz.net/name_enter/94063978
Added link

Added link: https://www.instiz.net/name_enter/94063674
Added link: https://www.instiz.net/name_enter/94063671
Added link: https://www.instiz.net/name_enter/94063669
Added link: https://www.instiz.net/name_enter/94063668
Added link: https://www.instiz.net/name_enter/94063667
Added link: https://www.instiz.net/name_enter/94063659
Added link: https://www.instiz.net/name_enter/94063656
Added link: https://www.instiz.net/name_enter/94063653
Added link: https://www.instiz.net/name_enter/94063652
Added link: https://www.instiz.net/name_enter/94063650
Added link: https://www.instiz.net/name_enter/94063646
Added link: https://www.instiz.net/name_enter/94063645
Added link: https://www.instiz.net/name_enter/94063644
Added link: https://www.instiz.net/name_enter/94063643
Added link: https://www.instiz.net/name_enter/94063642
Added link: https://www.instiz.net/name_enter/94063639
Added link: https://www.instiz.net/name_enter/94063638
Added link: https://www.instiz.net/name_enter/94063636
Added link

Added link: https://www.instiz.net/name_enter/94063328
Added link: https://www.instiz.net/name_enter/94063326
Added link: https://www.instiz.net/name_enter/94063325
Added link: https://www.instiz.net/name_enter/94063324
Added link: https://www.instiz.net/name_enter/94063320
Added link: https://www.instiz.net/name_enter/94063319
Added link: https://www.instiz.net/name_enter/94063318
Added link: https://www.instiz.net/name_enter/94063317
Added link: https://www.instiz.net/name_enter/94063316
Added link: https://www.instiz.net/name_enter/94063313
Added link: https://www.instiz.net/name_enter/94063312
Added link: https://www.instiz.net/name_enter/94063310
Added link: https://www.instiz.net/name_enter/94063308
Added link: https://www.instiz.net/name_enter/94063307
Added link: https://www.instiz.net/name_enter/94063306
Added link: https://www.instiz.net/name_enter/94063305
Added link: https://www.instiz.net/name_enter/94063304
Added link: https://www.instiz.net/name_enter/94063303
Added link

Added link: https://www.instiz.net/name_enter/94063009
Added link: https://www.instiz.net/name_enter/94063004
Added link: https://www.instiz.net/name_enter/94063001
Added link: https://www.instiz.net/name_enter/94063000
Added link: https://www.instiz.net/name_enter/94062996
Added link: https://www.instiz.net/name_enter/94062994
Added link: https://www.instiz.net/name_enter/94062992
Added link: https://www.instiz.net/name_enter/94062985
Added link: https://www.instiz.net/name_enter/94062983
Added link: https://www.instiz.net/name_enter/94062981
Added link: https://www.instiz.net/name_enter/94062979
Added link: https://www.instiz.net/name_enter/94062978
Added link: https://www.instiz.net/name_enter/94062977
Added link: https://www.instiz.net/name_enter/94062969
Added link: https://www.instiz.net/name_enter/94062967
Added link: https://www.instiz.net/name_enter/94062955
Added link: https://www.instiz.net/name_enter/94062954
Added link: https://www.instiz.net/name_enter/94062951
Added link

...

Added link: https://www.instiz.net/name_enter/94036836
Added link: https://www.instiz.net/name_enter/94036835
Added link: https://www.instiz.net/name_enter/94036834
Added link: https://www.instiz.net/name_enter/94036829
Added link: https://www.instiz.net/name_enter/94036828
Added link: https://www.instiz.net/name_enter/94036825
Added link: https://www.instiz.net/name_enter/94036821
Added link: https://www.instiz.net/name_enter/94036820
Added link: https://www.instiz.net/name_enter/94036816
Added link: https://www.instiz.net/name_enter/94036814
Added link: https://www.instiz.net/name_enter/94036813
Added link: https://www.instiz.net/name_enter/94036812
Added link: https://www.instiz.net/name_enter/94036808
Added link: https://www.instiz.net/name_enter/94036807
Added link: https://www.instiz.net/name_enter/94036803
Added link: https://www.instiz.net/name_enter/94036799
Added link: https://www.instiz.net/name_enter/94036798
Added link: https://www.instiz.net/name_enter/94036795
Added link

Added link: https://www.instiz.net/name_enter/94036455
Added link: https://www.instiz.net/name_enter/94036449
Added link: https://www.instiz.net/name_enter/94036442
Added link: https://www.instiz.net/name_enter/94036439
Added link: https://www.instiz.net/name_enter/94036436
Added link: https://www.instiz.net/name_enter/94036434
Added link: https://www.instiz.net/name_enter/94036430
Added link: https://www.instiz.net/name_enter/94036429
Added link: https://www.instiz.net/name_enter/94036426
Added link: https://www.instiz.net/name_enter/94036425
Added link: https://www.instiz.net/name_enter/94036423
Added link: https://www.instiz.net/name_enter/94036422
Added link: https://www.instiz.net/name_enter/94036420
Added link: https://www.instiz.net/name_enter/94036417
Added link: https://www.instiz.net/name_enter/94036415
Added link: https://www.instiz.net/name_enter/94036412
Added link: https://www.instiz.net/name_enter/94036410
Added link: https://www.instiz.net/name_enter/94036409
Added link

Added link: https://www.instiz.net/name_enter/94036025
Added link: https://www.instiz.net/name_enter/94036019
Added link: https://www.instiz.net/name_enter/94036018
Added link: https://www.instiz.net/name_enter/94036016
Added link: https://www.instiz.net/name_enter/94036014
Added link: https://www.instiz.net/name_enter/94036013
Added link: https://www.instiz.net/name_enter/94036010
Added link: https://www.instiz.net/name_enter/94036008
Added link: https://www.instiz.net/name_enter/94036007
Added link: https://www.instiz.net/name_enter/94036001
Added link: https://www.instiz.net/name_enter/94036000
Added link: https://www.instiz.net/name_enter/94035999
Added link: https://www.instiz.net/name_enter/94035998
Added link: https://www.instiz.net/name_enter/94035993
Added link: https://www.instiz.net/name_enter/94035991
Added link: https://www.instiz.net/name_enter/94035990
Added link: https://www.instiz.net/name_enter/94035988
Added link: https://www.instiz.net/name_enter/94035984
Added link

Added link: https://www.instiz.net/name_enter/94035415
Added link: https://www.instiz.net/name_enter/94035413
Added link: https://www.instiz.net/name_enter/94035411
Added link: https://www.instiz.net/name_enter/94035407
Added link: https://www.instiz.net/name_enter/94035401
Added link: https://www.instiz.net/name_enter/94035400
Added link: https://www.instiz.net/name_enter/94035390
Added link: https://www.instiz.net/name_enter/94035389
Added link: https://www.instiz.net/name_enter/94035386
Added link: https://www.instiz.net/name_enter/94035381
Added link: https://www.instiz.net/name_enter/94035378
Added link: https://www.instiz.net/name_enter/94035377
Added link: https://www.instiz.net/name_enter/94035372
Added link: https://www.instiz.net/name_enter/94035370
Added link: https://www.instiz.net/name_enter/94035368
Added link: https://www.instiz.net/name_enter/94035367
Added link: https://www.instiz.net/name_enter/94035365
Added link: https://www.instiz.net/name_enter/94035361
Added link

Added link: https://www.instiz.net/name_enter/94034580
Added link: https://www.instiz.net/name_enter/94034579
Added link: https://www.instiz.net/name_enter/94034575
Added link: https://www.instiz.net/name_enter/94034570
Added link: https://www.instiz.net/name_enter/94034568
Added link: https://www.instiz.net/name_enter/94034567
Added link: https://www.instiz.net/name_enter/94034557
Added link: https://www.instiz.net/name_enter/94034552
Added link: https://www.instiz.net/name_enter/94034550
Added link: https://www.instiz.net/name_enter/94034543
Added link: https://www.instiz.net/name_enter/94034542
Added link: https://www.instiz.net/name_enter/94034539
Added link: https://www.instiz.net/name_enter/94034533
Added link: https://www.instiz.net/name_enter/94034530
Added link: https://www.instiz.net/name_enter/94034526
Added link: https://www.instiz.net/name_enter/94034511
Added link: https://www.instiz.net/name_enter/94034509
Added link: https://www.instiz.net/name_enter/94034507
Added link

In [99]:
filename = "INSTIZ_CONTENTS.json"
existing_ids = set()
all_content = []

if os.path.exists(filename):
    with open(filename, "r", encoding="utf-8") as f:
        try:
            existing_content = json.load(f)
            existing_ids = {item['id'] for item in existing_content}  # 기존 JSON 파일의 post_id 수집
            all_content.extend(existing_content)  # 기존 데이터를 리스트에 추가
        except json.JSONDecodeError:
            pass  # 파일이 비어있거나 포맷이 잘못된 경우 무시하고 진행

In [101]:
# 새 데이터를 가져오고 기존 ID와 비교 후 추가
for idx, link in enumerate(info[:], start=1):

    try:
        content = get_instiz_content(link)
        if content['id'] not in existing_ids:  # 중복 확인
            all_content.append(content)
            existing_ids.add(content['id'])  # 중복 방지를 위해 추가된 ID를 기록
            print(f"[{idx}/{total_links}] Successfully processed: {link}")
        else:
            print(f"[{idx}/{total_links}] Duplicate ID, skipped: {content['id']}")
    except Exception as e:
        print(f"[{idx}/{total_links}] Error occurred while processing {link}: {e}")
        failed_links.append(link)  # 에러난 링크 저장
    
    if idx >= total_links:
        break
# 모든 데이터 처리 후 한 번만 저장
with open(filename, "w", encoding="utf-8") as f:
    json.dump(all_content, f, indent=4, ensure_ascii=False)
print(f"\nAll records saved to {filename} (Total records: {len(all_content)})")

# 실패한 링크 저장
if failed_links:
    with open("failed_links.json", "w", encoding="utf-8") as f:
        json.dump(failed_links, f, indent=4, ensure_ascii=False)
    print(f"Failed links saved to failed_links.json (Total failed: {len(failed_links)})")

print("json end")

[1/14308] Successfully processed: https://www.instiz.net/name_enter/94064362
[2/14308] Successfully processed: https://www.instiz.net/name_enter/94064358
[3/14308] Successfully processed: https://www.instiz.net/name_enter/94064347
[4/14308] Successfully processed: https://www.instiz.net/name_enter/94064345
[5/14308] Successfully processed: https://www.instiz.net/name_enter/94064343
[6/14308] Successfully processed: https://www.instiz.net/name_enter/94064342
[7/14308] Successfully processed: https://www.instiz.net/name_enter/94064340
[8/14308] Successfully processed: https://www.instiz.net/name_enter/94064339
[9/14308] Successfully processed: https://www.instiz.net/name_enter/94064333
[10/14308] Successfully processed: https://www.instiz.net/name_enter/94064332
[11/14308] Successfully processed: https://www.instiz.net/name_enter/94064330
[12/14308] Successfully processed: https://www.instiz.net/name_enter/94064327
[13/14308] Successfully processed: https://www.instiz.net/name_enter/9406

[107/14308] Successfully processed: https://www.instiz.net/name_enter/94064070
[108/14308] Successfully processed: https://www.instiz.net/name_enter/94064068
[109/14308] Successfully processed: https://www.instiz.net/name_enter/94064067
[110/14308] Successfully processed: https://www.instiz.net/name_enter/94064066
[111/14308] Successfully processed: https://www.instiz.net/name_enter/94064063
[112/14308] Successfully processed: https://www.instiz.net/name_enter/94064061
[113/14308] Successfully processed: https://www.instiz.net/name_enter/94064060
[114/14308] Successfully processed: https://www.instiz.net/name_enter/94064059
[115/14308] Successfully processed: https://www.instiz.net/name_enter/94064057
[116/14308] Successfully processed: https://www.instiz.net/name_enter/94064051
[117/14308] Successfully processed: https://www.instiz.net/name_enter/94064050
[118/14308] Successfully processed: https://www.instiz.net/name_enter/94064042
[119/14308] Successfully processed: https://www.inst

[211/14308] Successfully processed: https://www.instiz.net/name_enter/94063845
[212/14308] Successfully processed: https://www.instiz.net/name_enter/94063844
[213/14308] Successfully processed: https://www.instiz.net/name_enter/94063842
[214/14308] Successfully processed: https://www.instiz.net/name_enter/94063841
[215/14308] Successfully processed: https://www.instiz.net/name_enter/94063840
[216/14308] Successfully processed: https://www.instiz.net/name_enter/94063838
[217/14308] Successfully processed: https://www.instiz.net/name_enter/94063830
[218/14308] Successfully processed: https://www.instiz.net/name_enter/94063829
[219/14308] Successfully processed: https://www.instiz.net/name_enter/94063827
[220/14308] Successfully processed: https://www.instiz.net/name_enter/94063826
[221/14308] Successfully processed: https://www.instiz.net/name_enter/94063825
[222/14308] Successfully processed: https://www.instiz.net/name_enter/94063823
[223/14308] Successfully processed: https://www.inst

[315/14308] Successfully processed: https://www.instiz.net/name_enter/94063610
[316/14308] Successfully processed: https://www.instiz.net/name_enter/94063609
[317/14308] Successfully processed: https://www.instiz.net/name_enter/94063608
[318/14308] Successfully processed: https://www.instiz.net/name_enter/94063607
[319/14308] Successfully processed: https://www.instiz.net/name_enter/94063604
[320/14308] Successfully processed: https://www.instiz.net/name_enter/94063595
[321/14308] Successfully processed: https://www.instiz.net/name_enter/94063592
[322/14308] Successfully processed: https://www.instiz.net/name_enter/94063590
[323/14308] Successfully processed: https://www.instiz.net/name_enter/94063586
[324/14308] Successfully processed: https://www.instiz.net/name_enter/94063585
[325/14308] Successfully processed: https://www.instiz.net/name_enter/94063584
[326/14308] Successfully processed: https://www.instiz.net/name_enter/94063583
[327/14308] Successfully processed: https://www.inst

[419/14308] Successfully processed: https://www.instiz.net/name_enter/94063387
[420/14308] Successfully processed: https://www.instiz.net/name_enter/94063386
[421/14308] Successfully processed: https://www.instiz.net/name_enter/94063385
[422/14308] Successfully processed: https://www.instiz.net/name_enter/94063383
[423/14308] Successfully processed: https://www.instiz.net/name_enter/94063381
[424/14308] Successfully processed: https://www.instiz.net/name_enter/94063380
[425/14308] Successfully processed: https://www.instiz.net/name_enter/94063379
[426/14308] Successfully processed: https://www.instiz.net/name_enter/94063378
[427/14308] Successfully processed: https://www.instiz.net/name_enter/94063377
[428/14308] Successfully processed: https://www.instiz.net/name_enter/94063376
[429/14308] Successfully processed: https://www.instiz.net/name_enter/94063374
[430/14308] Successfully processed: https://www.instiz.net/name_enter/94063372
[431/14308] Successfully processed: https://www.inst

...

[13975/14308] Error occurred while processing https://www.instiz.net/name_enter/94035766: 403 Client Error: Forbidden for url: https://www.instiz.net/name_enter/94035766
[13976/14308] Error occurred while processing https://www.instiz.net/name_enter/94035759: 403 Client Error: Forbidden for url: https://www.instiz.net/name_enter/94035759
[13977/14308] Error occurred while processing https://www.instiz.net/name_enter/94035758: 403 Client Error: Forbidden for url: https://www.instiz.net/name_enter/94035758
[13978/14308] Error occurred while processing https://www.instiz.net/name_enter/94035757: 403 Client Error: Forbidden for url: https://www.instiz.net/name_enter/94035757
[13979/14308] Error occurred while processing https://www.instiz.net/name_enter/94035754: 403 Client Error: Forbidden for url: https://www.instiz.net/name_enter/94035754
[13980/14308] Error occurred while processing https://www.instiz.net/name_enter/94035750: 403 Client Error: Forbidden for url: https://www.instiz.net/

[14027/14308] Successfully processed: https://www.instiz.net/name_enter/94035550
[14028/14308] Error occurred while processing https://www.instiz.net/name_enter/94035539: 403 Client Error: Forbidden for url: https://www.instiz.net/name_enter/94035539
[14029/14308] Error occurred while processing https://www.instiz.net/name_enter/94035531: 403 Client Error: Forbidden for url: https://www.instiz.net/name_enter/94035531
[14030/14308] Successfully processed: https://www.instiz.net/name_enter/94035513
[14031/14308] Error occurred while processing https://www.instiz.net/name_enter/94035510: 403 Client Error: Forbidden for url: https://www.instiz.net/name_enter/94035510
[14032/14308] Error occurred while processing https://www.instiz.net/name_enter/94035506: 403 Client Error: Forbidden for url: https://www.instiz.net/name_enter/94035506
[14033/14308] Error occurred while processing https://www.instiz.net/name_enter/94035503: 403 Client Error: Forbidden for url: https://www.instiz.net/name_ent

[14093/14308] Successfully processed: https://www.instiz.net/name_enter/94035178
[14094/14308] Error occurred while processing https://www.instiz.net/name_enter/94035170: 403 Client Error: Forbidden for url: https://www.instiz.net/name_enter/94035170
[14095/14308] Error occurred while processing https://www.instiz.net/name_enter/94035163: 403 Client Error: Forbidden for url: https://www.instiz.net/name_enter/94035163
[14096/14308] Error occurred while processing https://www.instiz.net/name_enter/94035162: 403 Client Error: Forbidden for url: https://www.instiz.net/name_enter/94035162
[14097/14308] Error occurred while processing https://www.instiz.net/name_enter/94035161: 403 Client Error: Forbidden for url: https://www.instiz.net/name_enter/94035161
[14098/14308] Error occurred while processing https://www.instiz.net/name_enter/94035160: 403 Client Error: Forbidden for url: https://www.instiz.net/name_enter/94035160
[14099/14308] Successfully processed: https://www.instiz.net/name_ent

[14173/14308] Successfully processed: https://www.instiz.net/name_enter/94034782
[14174/14308] Successfully processed: https://www.instiz.net/name_enter/94034778
[14175/14308] Successfully processed: https://www.instiz.net/name_enter/94034777
[14176/14308] Successfully processed: https://www.instiz.net/name_enter/94034774
[14177/14308] Successfully processed: https://www.instiz.net/name_enter/94034771
[14178/14308] Successfully processed: https://www.instiz.net/name_enter/94034756
[14179/14308] Successfully processed: https://www.instiz.net/name_enter/94034755
[14180/14308] Successfully processed: https://www.instiz.net/name_enter/94034754
[14181/14308] Successfully processed: https://www.instiz.net/name_enter/94034753
[14182/14308] Successfully processed: https://www.instiz.net/name_enter/94034743
[14183/14308] Successfully processed: https://www.instiz.net/name_enter/94034735
[14184/14308] Successfully processed: https://www.instiz.net/name_enter/94034724
[14185/14308] Successfully p

[14275/14308] Successfully processed: https://www.instiz.net/name_enter/94034293
[14276/14308] Successfully processed: https://www.instiz.net/name_enter/94034289
[14277/14308] Successfully processed: https://www.instiz.net/name_enter/94034282
[14278/14308] Successfully processed: https://www.instiz.net/name_enter/94034278
[14279/14308] Successfully processed: https://www.instiz.net/name_enter/94034275
[14280/14308] Successfully processed: https://www.instiz.net/name_enter/94034267
[14281/14308] Successfully processed: https://www.instiz.net/name_enter/94034266
[14282/14308] Successfully processed: https://www.instiz.net/name_enter/94034264
[14283/14308] Successfully processed: https://www.instiz.net/name_enter/94034257
[14284/14308] Successfully processed: https://www.instiz.net/name_enter/94034256
[14285/14308] Successfully processed: https://www.instiz.net/name_enter/94034255
[14286/14308] Successfully processed: https://www.instiz.net/name_enter/94034252
[14287/14308] Successfully p

In [105]:
retry_failed_links("failed_links_retry.json", filename, existing_ids, all_content)

print("Retry process complete")

[Retry 1/2] Error occurred while processing https://www.instiz.net/name_enter/94053504: 'NoneType' object has no attribute 'contents'
[Retry 2/2] Error occurred while processing https://www.instiz.net/name_enter/94038044: 'NoneType' object has no attribute 'contents'

All records saved to INSTIZ_CONTENTS.json (Total records: 25893)
Retry failed links saved to failed_links_retry.json (Total failed: 2)
Retry process complete
